# Load File

In [2]:
import numpy as np
import pandas as pd
import win32com.client 
from openpyxl import load_workbook

def read_excel(filename):
    xlapp = win32com.client.Dispatch('Excel.Application')
    basic_information = xlapp.Workbooks.Open(
        filename,False, True, None,
        Password = 'NCCUAI')

    data_list = list(xlapp.Worksheets(2).UsedRange())
    data = pd.DataFrame(data_list[1:], columns = data_list[0])
    basic_information.Close()
    return data



In [2]:
filename = 'C:/Users/USER/Desktop/AIML/資料/00_業務員資料.xlsx'
data = read_excel(filename)
data.head()

,AGENT_ID,AGENT_TITLE,ISABLE,START_DATE,END_DATE,BIR_DATE,AGENT_AGE,ON_BOARD_AGE,SENIORITY,AGENT_SEX,UNIT_POST_CODE,CITY,DISTRICT,LOCATION_ID
0,6749.0,VRM,0.0,19880613.0,0.0,19581206.0,63.7,29.5,34.2,1.0,407667.0,台中市,西屯區,2.0
1,35172.0,AM,0.0,19940206.0,0.0,19641201.0,57.7,29.2,28.6,0.0,600578.0,嘉義市,西區,3.0
2,11789.0,UM,0.0,19890326.0,0.0,19620102.0,60.7,27.2,33.4,0.0,105608.0,台北市,松山區,1.0
3,36925.0,DM,0.0,19890416.0,0.0,19580405.0,64.4,31.0,33.4,0.0,220781.0,新北市,板橋區,1.0
4,25793.0,DM,0.0,19941006.0,0.0,19621008.0,59.9,32.0,27.9,0.0,407667.0,台中市,西屯區,2.0


In [6]:
filename = 'C:/Users/USER/Desktop/AIML/資料/07_壽險FYC.xlsx'
life_insurance = read_excel(filename)
life_insurance.head()

,AGENT_ID,YYYYMM,PREMIUM,PIECES,FYC
0,1.0,202109.0,0.0,0.0,0.0
1,1.0,202110.0,118814.5,5.5,40259.0
2,1.0,202111.0,2046694.0,3.0,18599.0
3,1.0,202112.0,85300.0,3.0,35960.0
4,1.0,202201.0,46717.0,3.0,2084.0


In [4]:
filename = 'C:/Users/USER/Desktop/AIML/資料/08_產險FYC.xlsx'
property_insurance = read_excel(filename)
property_insurance.head()

,AGENT_ID,YYYYMM,PROPERTY_INSURANCE_PERMIUM,PROPERTY_INSURANCE_FYC,PROPERTY_INSURANCE_PIECES
0,8.0,202109.0,32985.0,6535.0,11.0
1,8.0,202110.0,30702.0,6140.0,8.0
2,8.0,202111.0,26094.0,4810.0,9.0
3,8.0,202112.0,13101.0,2502.0,6.0
4,8.0,202201.0,23499.0,4700.0,6.0


# Visualisation

### Premium and pieces

In [10]:
print(life_insurance['PREMIUM'].mean())
print(life_insurance['FYC'].mean())
print(life_insurance['PIECES'].mean())
print(property_insurance['PROPERTY_INSURANCE_PERMIUM'].mean())
print(property_insurance['PROPERTY_INSURANCE_FYC'].mean())
print(property_insurance['PROPERTY_INSURANCE_PIECES'].mean())

150160.21081785954
9153.565625635661
1.0339806177897888
45073.196203613974
4495.850695434076
10.56304033897661


### On_board_year and premium

In [ ]:
import matplotlib.pyplot as plt

# Date
start_date = pd.to_datetime(data['START_DATE'], 
    format = '%Y%m%d', 
    errors = 'coerce')
start_year = []
for i in start_date:
    start_year.append(i.year)
data['START_YEAR'] = start_year

# property insurance mean
property_insurancemean = (property_insurance['PROPERTY_INSURANCE_PERMIUM']
  .groupby(property_insurance['AGENT_ID']).mean())

# life_insurancemean
life_insurancemean = (life_insurance['PREMIUM']
  .groupby(life_insurance['AGENT_ID']).mean())

# Combine property insurance mean and life_insurancemean
data['life_premium'] = ''
data['property_premium'] = ''
for k, i in enumerate(property_insurancemean.index):
    data['property_premium'][data['AGENT_ID'] == i] = property_insurancemean.iloc[k]
for k, i in enumerate(life_insurancemean.index):
    data['life_premium'][data['AGENT_ID'] == i] = life_insurancemean.iloc[k]

data.head()

In [ ]:
# Store data
import pickle

with open('C:/Users/USER/Desktop/機器學習與人工智慧實作/資料/Data_FYC', 'wb') as fi:
    pickle.dump(data, fi)

In [2]:
# Read data
import pickle
with open('C:/Users/USER/Desktop/機器學習與人工智慧實作/資料/Data_FYC', 'rb') as fi:
    data_ = pickle.load(fi)

data_.head(2)

,AGENT_ID,AGENT_TITLE,ISABLE,START_DATE,END_DATE,BIR_DATE,AGENT_AGE,ON_BOARD_AGE,SENIORITY,AGENT_SEX,UNIT_POST_CODE,CITY,DISTRICT,LOCATION_ID,life_premium,property_premium,AGENT_TITLE_ENCODE,START_YEAR
0,6749.0,VRM,0.0,19880613.0,0.0,19581206.0,63.7,29.5,34.2,1.0,407667.0,台中市,西屯區,2.0,,274.166667,22,1988
1,35172.0,AM,0.0,19940206.0,0.0,19641201.0,57.7,29.2,28.6,0.0,600578.0,嘉義市,西區,3.0,,42202.416667,0,1994


In [ ]:
for k, i in enumerate(data_['property_premium']):
    if i == '':
        data_['property_premium'].iloc[k] = 0
a = (data_['property_premium']
    .groupby(data_['START_YEAR']).mean())

plt.plot(a.index, a.iloc[:])

In [ ]:
for k, i in enumerate(data_['life_premium']):
    if i == '':
        data_['life_premium'].iloc[k] = 0
a = (data_['life_premium']
    .groupby(data_['START_YEAR']).mean())
    
plt.plot(a.index, a.iloc[:])